In [54]:
import transformers
from datasets import load_dataset
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import numpy as np
import spacy
from spacy.util import filter_spans
from spacy.tokens import DocBin, Doc


In [7]:
dataset = load_dataset("ade_corpus_v2",'Ade_corpus_v2_drug_ade_relation')

  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'effect', 'indexes'],
        num_rows: 6821
    })
})


In [8]:
dataset['train'][0]

{'text': 'Intravenous azithromycin-induced ototoxicity.',
 'drug': 'azithromycin',
 'effect': 'ototoxicity',
 'indexes': {'drug': {'start_char': [12], 'end_char': [24]},
  'effect': {'start_char': [33], 'end_char': [44]}}}

In [34]:
training_dataset = {}

for row in dataset['train']:
    start_drug = row["indexes"]["drug"]["start_char"]
    end_drug = row["indexes"]["drug"]["end_char"]
    start_effect = row["indexes"]["effect"]["start_char"]
    end_effect = row["indexes"]["effect"]["end_char"]
    if row['text'] not in training_dataset:
        training_dataset[row['text']] = {}
        training_dataset[row['text']]['entities'] = set()
    
    for i,drug_start_index in enumerate(start_drug):
        training_dataset[row['text']]['entities'].add((start_drug[i], end_drug[i], "DRUG"))
    for i,effect_start_index in enumerate(start_effect):
        training_dataset[row['text']]['entities'].add((start_effect[i], end_effect[i], "REACTION"))
        
                                                            
   

In [35]:
#Convert dictionary to list of dictionaries
training_dataset = [{'text': key, 'entities': list(value['entities'])} for key, value in training_dataset.items()]

In [37]:
training_dataset[0]

{'text': 'Intravenous azithromycin-induced ototoxicity.',
 'entities': [(12, 24, 'DRUG'), (33, 44, 'REACTION')]}

In [ ]:
#Find longest text in the dataset in words

In [36]:
nlp = spacy.blank("en")

In [39]:
def to_doc_format(row):
    text = row['text']
    annotations = row['entities']
    
    doc = nlp.make_doc(text)
    ents = []
    
    #Remove text key from row
    gold_dict = row.copy()
    del gold_dict["text"]

    #example = Example.from_dict(doc, gold_dict)
    for start, end, label in gold_dict["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="expand")
        if span is None:
            print(start, end, label)
            print(doc.text[start:end])
            print(text)
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    
    return(doc)

In [40]:
docs = []
for row in training_dataset:
    docs.append(to_doc_format(row))


In [66]:
max = 0
for doc in docs :
    lenght = len([token.text for token in doc])
    if lenght > max:
        max = lenght
print(max)

97


In [51]:
#Mix the data randomly
np.random.shuffle(docs)

In [53]:
split_index = int(0.8 * len(docs))

TRAIN_DATA = docs[:split_index]
TEST_DATA = docs[split_index:]

In [55]:
train_bin = DocBin(docs=TRAIN_DATA)
test_bin = DocBin(docs=TEST_DATA)

In [56]:
train_bin.to_disk("train.spacy")
test_bin.to_disk("test.spacy")

In [57]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
